# WiNDC National Model

You will need to point to the correct locations for a few objects, listed below

In [1]:
parameter_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\DAAData.jld2"
set_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\Indices.jld2";

In [2]:
using MPSGE_MP

using CSV, DataFrames, JLD2

using JuMP

In [3]:
P= load(parameter_path)["data"] # load in date from saved Notebook output Dict, named P
S= load(set_path)["data"] # load in date from saved Notebook output Dict, named P


I = [i for i∈S[:i] if i∉[:use,:oth]]
J = [i for i∈S[:j] if i∉[:use,:oth]]
VA = [va for va∈S[:va] if va!=:othtax]
FD = S[:fd]
TS = S[:ts]
YR = S[:yr]
M = S[:m]

a_0 = P[:a_0]
id_0 = P[:id_0]
ys_0 = P[:ys_0]
tm_0 = P[:tm_0]
va_0 = P[:va_0]
md_0 = P[:md_0]
fd_0 = P[:fd_0]
m_0 = P[:m_0]
ty_0 = P[:ty_0]
ms_0 = P[:ms_0]
bopdef_0 = P[:bopdef_0]
x_0 = P[:x_0]
ta_0 = P[:ta_0]
#s_0 = P[:s_0]
fs_0 = P[:fs_0]
y_0 = P[:y_0];


ta = deepcopy(ta_0) *0
ty = deepcopy(ty_0)
tm = deepcopy(tm_0) *0;

1;

In [9]:
yr = Symbol(2017)

WiNnat = MPSGEModel()

#y_ = [j for j∈J if sum(ys_0[yr,j,i] for i∈I) !=0]
#a_ = [i_ for i_∈I if a_0[yr,i_]!=0]

@sectors(WiNnat,begin
    Y, (index = [J], description = "Sectoral Production")
    A, (index = [I], description = "Armington Supply")
    MS, (index = [M],description = "Margin Supply")
end)

@commodities(WiNnat,begin
    PA, (index = [I],  description = "Armington Price")
    PY, (index = [J],  description = "Supply")
    PVA, (index = [VA],description = "Value-added")
    PM, (index = [M],  description = "Foreign Exachange")
    PFX
end)

@consumer(WiNnat, RA, description = "Representative Agent")


for j∈J
    add_production!(WiNnat, Y[j],
        ScalarNest(:t; elasticity = 0, children = [
            ScalarOutput(PY[i],ys_0[yr,j,i]; taxes = [Tax(RA,ty[yr,j])]) for i∈I
        ]),
        ScalarNest(:s; elasticity = 0, children = [
            [ScalarInput(PA[i], id_0[yr,i,j]) for i∈I]; 
            [ScalarNest(:va; elasticity =  1, children = 
                [ScalarInput(PVA[va], va_0[yr,va,j]) for va∈VA])
            ];
        ])
    )
end


for m∈M
    add_production!(WiNnat, MS[m],
        ScalarNest(:t; elasticity = 0, children = [
            ScalarOutput(PM[m], sum(ms_0[yr,i,m] for i∈I))
        ]),
        ScalarNest(:s; elasticity = 0, children = [
            ScalarInput(PY[i], ms_0[yr,i,m]) for i∈I
        ])
    )
end


for i∈I
    add_production!(WiNnat, A[i],
        ScalarNest(:t; elasticity=2, children = [
            [ScalarOutput(PA[i], a_0[yr,i], taxes=[Tax(RA,ta[yr,i])],reference_price=1-ta_0[yr,i])];
            [ScalarOutput(PFX, x_0[yr,i])]
        ]),
        ScalarNest(:s; elasticity=0, children = [
            [ScalarInput(PM[m], md_0[yr,m,i]) for m∈M];
            ScalarNest(:dm; elasticity = 2, children = [
                ScalarInput(PY[i], y_0[yr,i]),
                ScalarInput(PFX, m_0[yr,i], taxes = [Tax(RA,tm[yr,i])],reference_price=1+tm_0[yr,i])
            ])
        ])
    )
end


add_demand!(WiNnat, RA,
    [ScalarDem(PA[i], fd_0[yr,i,:pce]) for i∈I],
    [
        [ScalarEndowment(PY[i], fs_0[yr,i]) for i∈I];
        [ScalarEndowment(PFX, bopdef_0[yr])];
        [ScalarEndowment(PA[i], -sum(fd_0[yr,i,xfd] for xfd∈FD if xfd!=:pce)) for i∈I];
        [ScalarEndowment(PVA[va], sum(va_0[yr,va,j] for j∈J)) for va∈VA]
    ]
);

In [10]:
jm = build!(WiNnat)

A JuMP Model
Feasibility problem with:
Variables: 292
`Vector{NonlinearExpr}`-in-`MathOptInterface.Complements`: 282 constraints
`Vector{AffExpr}`-in-`MathOptInterface.Complements`: 7 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 290 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Path 5.0.03
Names registered in the model: A, MS, PA, PFX, PM, PVA, PY, RA, Y, income_balance, market_clearance, zero_profit

In [11]:
fix(RA,12453.896315446877)

solve!(WiNnat; :cumulative_iteration_limit => 10_000)

df = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_4FE.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Zero:     3 Single:     0 Double:     0
Preprocessed size   : 288

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             3.7941e+02             0.0e+00 (f[  155])
    1     1     0   288 1.9248e+01  1.0e+00    3.8e+00 (f[  292])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.9248e+01           I 1.9e-01 1.9e+01 (f[  292])
    1     1     3     3 5.3145e-01  1.0e+00 SO 7.7e-02 4.2e-01 (f[  292])
    2     1     4     4 4.7337e-03  1.0e+00 SO 3.1e-02 3.2e-03 (f[  116])
    3     1     5     5 5.6381e-05  1.0e+00 SO 4.7e-04 4.8e-05 (f[  187])
    4     1     6     6 6.4209e-09  1.0e+00 SO 5.6e-

In [12]:
df |>
    x -> sort(x, :margin, rev=true)

Row,var,value,margin
,GenericV…,Float64,Float64
1,PA[pip],0.779861,5.36439e-9
2,RA,12453.9,2.09184e-10
3,PVA[compen],0.9916,3.27418e-11
4,PA[pet],0.820925,1.3074e-11
5,PY[oil],0.970186,1.06866e-11
6,PY[mch],0.965957,6.59384e-12
7,PA[mot],0.929959,4.77485e-12
8,PY[pet],0.95076,2.6148e-12
9,PA[rnt],0.932068,2.359e-12
